[#air-polution-sensor](https://github.com/Johansmm/air-polution-sensor)

 # Spatial and temporal analysis of signals to detect drift of air quality sensors
 
The network of sensors allows to follow a complex phenomenon by observing the temporal evolution of the values in several points, and by crossing the information from one sensor to another. This allows, for example, to carry out a meteorological or seismic monitoring, by detecting a cloud or a tremor. However, the sensors used are sometimes subject to drift, wear and tear or possible interference, which makes some of the observed values false. It is therefore important to be able to determine if a sensor starts to drift in order to allow a good analysis of the data. Therefore, the objective of this project is to detect a drifting sensor (when and where) in a real sensor's network. For this, we will explore the usefulness of a particular mathematical tool: the graph space-time spectrogram. Such a tool allows to decompose a space-time series into a sum of space-time frequencies, in a similar way to Fourier analysis, but for signals in graphs.

In [ ]:
# Libraries
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Basis libraries 
import sys, getpass, os, copy
import numpy as np
import scipy.signal
from matplotlib import rc
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import scipy.signal
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from statistics import mean 

# Library for boxplots
# font = {'family':'sans-serif','sans-serif':['Helvetica'], 'size':18}
# rc('font', **font)
# rc('text', usetex=True)

np.random.seed(1)

In [ ]:
# Colab access
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

# If you work-directory don't match, please add the new path to pwd_list
cwd_list = ["/content/gdrive/My Drive/S5 Project: Air polution/air-polution-sensor/",
            "/content/gdrive/My Drive/Colab Notebooks/ProyectS5/air-polution-sensor/",
            "/content/gdrive/MyDrive/IMT Atlantique/Project_S5_Air/air-polution-sensor/",
            os.path.join(os.getcwd(),"air-polution-sensor")]

for cwd in cwd_list:
    if os.path.isdir(cwd):
        %cd "$cwd"
        break
if os.path.split(os.getcwd())[-1] != "air-polution-sensor":
    print("[WARNING]: Incorrect working directory. Please add the fix-directory to the cwd_list.")
else: 
    print("[INFO]: Work directory: " + os.getcwd())

## **Download database**
Initially, the project [air-polution-sensor](https://github.com/Johansmm/air-polution-sensor.git) contains some default databases, which must be decompressed. Additionally, a larger database should be downloaded. This process is presented below.

Note: Make sure you are inside the repository folder [air-polution-sensor](https://github.com/Johansmm/air-polution-sensor.git).

In [ ]:
if not os.path.isdir("./data/origin_data"):
    if not os.path.isdir("./data"): os.mkdir("./data")
    if not os.path.isdir("./data/origin_data"): os.mkdir("./data/origin_data")
    !wget --output-document="./data/data.zip" "https://www.dropbox.com/sh/w1704rg9fd9z4pq/AABG3YCUMHTwbFhf3pKgti-Qa/PuneData_UseThis/August2019?dl=0&subfolder_nav_tracking=1"
    !unzip "./data/data.zip" -d "./data/origin_data"

if not os.path.isfile("./data/super_df.csv") and os.path.isfile("./data/super_df.rar"):
    !unrar x "./data/super_df.rar" "./data/"
    
if not os.path.isfile("./data/drift_joint_drift.csv") and os.path.isfile("./data/drift_joint_drift.rar"):
    !unrar x "./data/drift_joint_drift.rar" "./data/"
    
if not os.path.isfile("./data/drift_joint_real.csv") and os.path.isfile("./data/drift_joint_real.rar"):
    !unrar x "./data/drift_joint_real.rar" "./data/"

clear_output(wait = True)
print("[INFO]: Data downloaded successfully!")

In [ ]:
# Specific libraries
!pip install -r requirements.txt
from libraries.global_functions import *
from libraries.animation_utils import *
np.random.seed(0)

clear_output(wait = True)
print("[INFO]: Successfully loaded libraries!")

In [ ]:
if IN_COLAB:
    !sudo apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    clear_output(wait = True)
else:
    print("[INFO]: If you have problems, remember execute the following code:\n>> sudo apt-get install python3-dev graphviz libgraphviz-dev pkg-config")

## **Handling of NAN Values**


In order to obtain a good representation in the spectrogram is necessary to tame the NAN values that have the data provided by the sensors, for this reason we used a python library called fancyimpute that is very useful to perform data imputation, with different algorithms such as matrix completion, matrix completion by iterative low-rank SVD decomposition or nuclear norm minimization.

### Test in interval of the time series with fancyimpute


First, the dataframe is read where all the sensor data is contained in the time intervals that are interesting to observe.

In [ ]:
super_df = pd.read_csv('./data/superdfi.csv', header=0, index_col=0)


A dataframe sample is taken from the data of three sensors and the NAN values are removed.

In [ ]:
df_nonan = super_df[(super_df['Sensor'] == 'sensor14v2.csv') | (super_df['Sensor'] == 'sensor13v2.csv') | (super_df['Sensor'] == 'sensor12v2.csv') | (super_df['Sensor'] == 'sensor11v2.csv') | (super_df['Sensor'] == 'sensor10v2.csv') | (super_df['Sensor'] == 'sensor9v2.csv') | (super_df['Sensor'] == 'sensor8v2.csv') | (super_df['Sensor'] == 'sensor7v2.csv') ]

Due to the fact that the original data has NAN values, for the performance evaluation experiment these values will be excluded to pass a matrix without null values through the models and to be able to apply the performance indicators (MAE, MSE).

In [ ]:
aux = pd.pivot_table(data=df_nonan,values='PM2_MOY',index='Sensor',columns=['Date', 'Hour'] )
aux = aux.dropna(axis=1)
columnstaux = aux.columns
indextaux = aux.index

In [ ]:
aux = aux.T.reset_index().drop(columns=['Date', 'Hour']).T
aux

We transform the df into a numpy vector and randomly replace existing values with NAN values, we also create a mask of the positions of these values for later comparison.

In [ ]:
aux_arr = aux.copy(deep=True)
df_nonanArr = aux_arr.to_numpy()
arr_incomplete =  np.copy(df_nonanArr)
missing_raw_values = np.random.uniform(0, 1, df_nonanArr.shape)
missing_mask = missing_raw_values < 0.2
arr_incomplete[missing_mask] = np.nan

### Functions

In [ ]:
def reconstruction_error(XY, XY_completed, missing_mask, name=None):
    """
    Returns mean squared error and mean absolute error for
    completed matrices.
    """
    value_pairs = [
        (i, j, XY[i, j], XY_completed[i, j])
        for i in range(XY.shape[0])
        for j in range(XY.shape[1])
        if missing_mask[i, j]
    ]
    print("First 10 reconstructed values:")
    for (i, j, x, xr) in value_pairs[:10]:
        print("  (%d,%d)  %0.4f ~= %0.4f" % (i, j, x, xr))
    diffs = [actual - predicted for (_, _, actual, predicted) in value_pairs]
    missing_mse = np.sqrt(np.mean([diff ** 2 for diff in diffs]))
    missing_mae = np.mean([np.abs(diff) for diff in diffs])
    print("%sRMSE: %0.4f, MAE: %0.4f" % (
        "" if not name else name + " ",
        missing_mse,
        missing_mae))
    return missing_mse, missing_mae

In [ ]:
def plotTS (observed_data, imputed_data, sensor, title):

  s = sensor.split('.')[0]
  s=s[0:8] if len(s) == 10 else s[0:7] 
  plt.figure(figsize=(20,10))
  plt.subplot(211)
  plt.plot(observed_data[sensor],'r')
  plt.title('Original data, ' + s, size=20)
  plt.xlabel('Time',size=18)
  plt.ylabel('Average PM2.5',size=18)
  plt.subplots_adjust(hspace=0.4)
  
  #plt.figure(figsize=(20,5))
  plt.subplot(212)
  plt.plot(imputed_data[sensor],'b')
  plt.title(title + ', ' + s, size=20)
  plt.xlabel('Time',size=18)
  plt.ylabel('Average PM2.5',size=18)

  plt.show()



In [ ]:
def scatter(x,y,title):

  fig, ax = plt.subplots()
  ax.scatter(x, y, c='blue')
  line = mlines.Line2D([0, 1], [0, 1], color='black')
  transform = ax.transAxes
  line.set_transform(transform)
  ax.add_line(line)
  plt.title(title)
  plt.xlabel('Observed value')
  plt.ylabel('Imputed value')
  plt.show()

###  Matrix completion by iterative low-rank SVD decomposition

The following function will be in charge of evaluating the performance of the model.

In [ ]:
from fancyimpute import IterativeSVD

def test_iterative_svd(i):
    solver = IterativeSVD(rank=1, max_iters=400)
    arr_completed = solver.fit_transform(arr_incomplete[:,i:i+25])
    rmse, missing_mae = reconstruction_error(
        df_nonanArr[:,i:i + 25],
        arr_completed,
        missing_mask[:,i:i + 25],
        name="IterativeSVD")
    
    return arr_completed, rmse, missing_mae

arr = np.copy(arr_incomplete)
lrmseSVD = []
lmaeSVD = []
for i in range(0, len(arr_incomplete[1]), 25):
  arr_completedSVD, rmseSVD, maeSVD = test_iterative_svd(i)
  arr[:,i:i+25]= arr_completedSVD
  lrmseSVD.append(rmseSVD)
  lmaeSVD.append(maeSVD)

In [ ]:
print(mean(lrmseSVD), mean(lmaeSVD))

The resulting matrix is converted back into df 

In [ ]:
arr = pd.DataFrame(arr, columns = columnstaux, index= indextaux)


Then two graphs are made, the first is the original data and the second data resulting from the allocation of the model.

In [ ]:
plotTS(aux.T.reset_index(), arr.T.reset_index(), 'sensor14v2.csv', title= 'Average PM2.5 filled by iterative low-rank SVD decomposition,when the percentage of missing values equals 30%')

In [ ]:
# Inputed Value vs Observed value

x = aux.copy(deep=True)
x = x.to_numpy()
x = x[missing_mask]
y = arr.drop(columns=['Date', 'Hour'])
y = y.to_numpy()[missing_mask]
scatter(x, y, 'SVD')

### Softer Imputer

Next we use another model of the fancyimputer library that is also based on matrix completion, to be able to compare it with the previous one.

In [ ]:
from fancyimpute import SoftImpute

def test_soft_impute_with_low_rank_random_matrix(i):
    solver = SoftImpute()
    arr_completed = solver.fit_transform(arr_incomplete[i:i+100])
    rmse, missing_mae = reconstruction_error(
        df_nonanArr[i:i+100],
        arr_completed,
        missing_mask[i:i+100],
        name="SoftImpute")
    #assert missing_mae < 0.1, "Error too high!"
    return arr_completed, rmse, missing_mae

arr1 = np.copy(arr_incomplete)
lrmseSI = []
lmaeSI = []
for i in range(0, len(arr_incomplete), 100):
  arr_completedSI, rmseSI, maeSI = test_soft_impute_with_low_rank_random_matrix(i)
  arr1[i:i+100,:]= arr_completedSI
  lrmseSI.append(rmseSI)
  lmaeSI.append(maeSI)

We extract the mean of the rmse and mae performance indicators for each 100 points

In [ ]:
mean(lrmseSI), mean(lmaeSI)

In [ ]:
arr1 = pd.DataFrame(arr1, columns = columnstaux, index= indextaux).reset_index()
#arr_incomplete = pd.DataFrame(arr_incomplete, columns = columnstaux, index= indextaux).reset_index()

Below you can see the comparison of the graphs with predicted values vs. the original of the sensor 11.

In [ ]:
plotTS(aux.reset_index(), arr1, 'sensor11v2.csv', title= 'Average PM2.5 filled by soft imputation, when the percentage of missing values equals 30%')

In [ ]:
# Inputed Value vs Observed value

x = aux.copy(deep=True)
x = x.to_numpy()
x = x[missing_mask]
y = arr1.drop(columns=['Date', 'Hour'])
y = y.to_numpy()[missing_mask]
scatter(x,y, 'Soft Imputations')

### Matrix Completion Test

In [ ]:
from fancyimpute import MatrixFactorization

def fill_mf_test(df,i):
    solver = MatrixFactorization(rank= 4, optimization_algorithm='Adam', patience=15)
    arr_completed = solver.fit_transform(df[i:i+100])
    rmse, missing_mae = reconstruction_error(
    df_nonanArr[i:i+100],
    arr_completed,
    missing_mask[i:i+100],
    name="SoftImpute")
    return arr_completed , rmse, missing_mae


arr2 = np.copy(arr_incomplete)
lrmseMF = []
lmaeMF = []
for i in range(0, len(arr_incomplete), 100):
  arr_completedMF, rmseMF, maeMF = fill_mf_test(arr_incomplete,i)
  arr2[i:i+100,:]= arr_completedMF
  lrmseMF.append(rmseMF)
  lmaeMF.append(maeMF)

In [ ]:
print(mean(lrmseMF), mean(lmaeMF))

In [ ]:
arr2 = pd.DataFrame(arr2, columns = columnstaux, index= indextaux).reset_index()

In [ ]:
plotTS(aux.reset_index(), arr2, 'sensor11v2.csv', title= 'Average PM2.5 filled by matrix completion')

In [ ]:
# Inputed Value vs Observed value

x = aux.copy(deep=True)
x = x.to_numpy()
x = x[missing_mask]
y = arr2.drop(columns=['Date', 'Hour'])
y = y.to_numpy()[missing_mask]
scatter(x,y, 'Matrix Completion')

### Fill of data SVD

In [ ]:
super_df = pd.read_csv('./data/superdfi.csv', header=0, index_col=0)
originalwithNAN = pd.pivot_table(data=super_df,values='PM2_MOY' ,index=['Date', 'Hour'],columns='Sensor' ).reset_index()

In [ ]:
    filldf = super_df.copy(deep=True)
    filldf= pd.pivot_table(data=filldf,values='PM2_MOY' ,index=['Date', 'Hour'],columns='Sensor' )
    columnstf = filldf.columns
    indextf = filldf.index

In [ ]:
from fancyimpute import IterativeSVD

def fill_iterative_svd(df, predicted_gas, i):
    filldf = df.copy(deep=True)
    filldf= pd.pivot_table(data=filldf,values=predicted_gas ,index=['Date', 'Hour'],columns='Sensor' )
    columnstf = filldf.columns
    indextf = filldf.index
    X_incomplete = filldf.to_numpy()
    solver = IterativeSVD(rank=1, convergence_threshold=0.00000001,max_iters=400)
    arr_completed = solver.fit_transform(X_incomplete[i:i+100])
    return arr_completed, columnstf, indextf


aux1 = np.zeros((1394, 43))
aux2 = np.zeros((1394, 43))
aux3 = np.zeros((1394, 43))
for i in range(0, 1394, 100):
  arr_completedSVDPM2Min, columnstf, indextf = fill_iterative_svd(super_df,'PM2_MIN',i)
  arr_completedSVDPM2Max, columnstf, indextf = fill_iterative_svd(super_df, 'PM2_MAX',i)
  arr_completedSVDPM2Avg, columnstf, indextf = fill_iterative_svd(super_df, 'PM2_MOY',i)
  aux1[i:i+100,:]= arr_completedSVDPM2Min
  aux2[i:i+100,:]= arr_completedSVDPM2Max
  aux3[i:i+100,:]= arr_completedSVDPM2Avg


In [ ]:
aux3 = pd.DataFrame(aux3, columns = columnstf, index= indextf).reset_index()

In [ ]:
plotTS(originalwithNAN, aux3, 'sensor23v2.csv', title= 'Average PM2.5 filled by matrix completion by iterative low-rank SVD decomposition')

In [ ]:
# print(arr_completedSVDPM2Avg.isnull().sum().sum())
# print(arr_completedSVDPM2Min.isnull().sum().sum())
# print(arr_completedSVDPM2Max.isnull().sum().sum())

In [ ]:
# for idx in arr_completedSVDPM2Avg.columns:
#   if idx is not 'Date' and idx is not 'Hour':
#     dfFilled = pd.DataFrame()
#     dfFilled[['Date', 'Hour', 'PM2_MIN']] = arr_completedSVDPM2_Min[['Date', 'Hour', idx]]
#     dfFilled['PM2_MAX'] = arr_completedSVDPM2_Max[idx]
#     dfFilled['PM2_AVG'] = arr_completedSVDPM2Avg[idx]
#     dfFilled.to_csv('/content/gdrive/MyDrive/IMT Atlantique/Project_S5_Air/air-polution-sensor/data/data_filled/' + idx)

### Fill of Data soft imputation

In [ ]:
from fancyimpute import SoftImpute

def fill_iterative_softimpute(df, predicted_gas, i):
    filldf = df.copy(deep=True)
    filldf= pd.pivot_table(data=filldf,values=predicted_gas ,index=['Date', 'Hour'],columns='Sensor' )
    columnstf = filldf.columns
    indextf = filldf.index
    X_incomplete = filldf.to_numpy()
    solver = SoftImpute()
    arr_completed = solver.fit_transform(X_incomplete[i:i+100])
    return arr_completed , columnstf, indextf


aux1 = np.zeros((1394, 43))
aux2 = np.zeros((1394, 43))
aux3 = np.zeros((1394, 43))
for i in range(0, 1394, 100):
  arr_completedSIPM2Min, columnstf, indextf = fill_iterative_softimpute(super_df,'PM2_MIN',i)
  arr_completedSIPM2Max, columnstf, indextf = fill_iterative_softimpute(super_df, 'PM2_MAX',i)
  arr_completedSIPM2Avg, columnstf, indextf = fill_iterative_softimpute(super_df, 'PM2_MOY',i)
  aux1[i:i+100,:]= arr_completedSIPM2Min
  aux2[i:i+100,:]= arr_completedSIPM2Max
  aux3[i:i+100,:]= arr_completedSIPM2Avg

In [ ]:
aux3 = pd.DataFrame(aux3, columns = columnstf, index= indextf).reset_index()

In [ ]:
plotTS(originalwithNAN, aux3, 'sensor45v2.csv', title= 'Average PM2.5 filled by soft imputation')

### Fill of data Matrix completion

In [ ]:
from fancyimpute import MatrixFactorization

def fill_iterative_mf(df, predicted_gas,i):
    filldf = df.copy(deep=True)
    filldf= pd.pivot_table(data=filldf,values=predicted_gas ,index=['Date', 'Hour'],columns='Sensor' )
    columnstf = filldf.columns
    indextf = filldf.index
    X_incomplete = filldf.to_numpy()
    solver = MatrixFactorization(rank= 3, optimization_algorithm='Adam')
    arr_completed = solver.fit_transform(X_incomplete[i:i+100])
    
    return arr_completed
  

aux1 = np.zeros((1394, 43))
aux2 = np.zeros((1394, 43))
aux3MF = np.zeros((1394, 43))
for i in range(0, 1394, 100):
  #arr_completedMFPM2Min = fill_iterative_mf(super_df,'PM2_MIN',i)
  #arr_completedMFPM2Max = fill_iterative_mf(super_df, 'PM2_MAX',i)
  arr_completedMFPM2Avg = fill_iterative_mf(super_df, 'PM2_MOY',i)
  #aux1[i:i+100,:]= arr_completedMFPM2Min
  #aux2[i:i+100,:]= arr_completedMFPM2Max
  aux3MF[i:i+100,:]= arr_completedMFPM2Avg
  print(i)

In [ ]:
aux3MF = pd.DataFrame(aux3MF, columns = columnstf, index= indextf).reset_index()

In [ ]:
plotTS(originalwithNAN, aux3MF, 'sensor15v2.csv', title= 'Average PM2.5 filled by Matrix Completion')

In [ ]:
plotTS(originalwithNAN, aux3MF, 'sensor38v2.csv', title= 'Average PM2.5 filled by Matrix Completion')

In [ ]:
plotTS(originalwithNAN, aux3MF, 'sensor45v2.csv', title= 'Average PM2.5 filled by Matrix Completion')

## **Network implementation**

In this section the implementation of the network is made based on real information about the physical configuration of the sensor network, thanks to the adjacency matrix of the distances between sensors. On the other hand the analysis of the network behavior is made based on a simulated signal of the measurement of the sensors in differents instants in time

### Graph construction
In this section it is described how from real information about the location of the sensors a graph can be built

There is information on the distance between sensors, this is between 0 and 20 with this information we proceed to the construction of the final network. Taking into account that it is presented to reach a simple representation of the graph

In [ ]:
W = pd.read_csv('data/Pune_SensorLocationDistances.csv',header=0).set_index('0')
W.describe()

In [ ]:
Indices = [ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 21,
            22, 23, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40,
            41, 42, 43, 45, 46, 48, 50, 51, 59]
W.columns = [int(x) for x in W.columns]
W= W.loc[Indices,Indices]
W.shape
len(Indices)

In [ ]:
fig = px.imshow(W)
fig.update_layout(
    title='Adjacent matrix')
fig.show()

The following chart shows the probability distribution function (PDF) and the cumulative distribution function (CDF) of the distances in order to analyze and choose the limit to establish the weight between connections. It is observed that the average value of the connections is between 5 and 7 and there are few connections greater than 17. If we take a threshold of 5 that is, connections greater than this value would be eliminated, 50% of the existing connections would be maintained, and we focus on the sensors that have a certain closeness in space.

In [ ]:
fig = go.Figure(data=[go.Histogram(x=W.to_numpy().flatten(),nbinsx=12,histnorm='probability density')])
fig.update_layout(title = "PDF Distances",xaxis_title="Distances")
fig.show()

fig = go.Figure(data=[go.Histogram(x=W.to_numpy().flatten(),nbinsx=12,histnorm='probability density',cumulative_enabled=True)])
fig.update_layout(title = "CDF Distances",xaxis_title="Distances")
fig.show()

To define the weight of an edge connecting two vertices is used the thresholded Gaussian kernel weighting function given by :


$ W_{i,j} = \left\{\begin{matrix}
exp(- \frac{\left | dist(i,j) \right |^{^2}}{2 \theta ^{2}}) &  dist(i,j) \leq k \\ 
0 & otherwise
\end{matrix}\right.$





In [ ]:
Theta, k = 4, 6 #Choosen values of k and theta
W_normal = Norm_W(W.copy(),Theta,k) 
A=W_normal.values

In [ ]:
fig = px.imshow(W_normal)
fig.update_layout(title='Adjacent matrix')
fig.show()

The following chart shows the probability distribution function (PDF) and the cumulative distribution function (CDF) of the weighted matrix. Most of the connections are between 0 and 0.4, i.e. they have a low weight.

In [ ]:
fig = go.Figure(data=[go.Histogram(x=W_normal.to_numpy().flatten(),nbinsx=12,histnorm='probability density')])
fig.update_layout(title = "PDF Distances",xaxis_title="Distances")
fig.show()

fig = go.Figure(data=[go.Histogram(x=W_normal.to_numpy().flatten(),nbinsx=12,histnorm='probability density',cumulative_enabled=True)])
fig.update_layout(title = "CDF Distances",xaxis_title="Distances")
fig.show()

The following graph shows the network with the original values of the adjacent matrix. 

In [ ]:
G = create_graph(W)
plot_graph(G)

In [ ]:
W_aux=W.copy()
column=W_aux.columns.to_list()
W_aux[column] = W_aux[column].where(~(W_aux[column]>10),other=0)

The following chart shows the W chart limiting the network connections

In [ ]:
G = create_graph(W_aux)
plot_graph(G)

A algotirm is applied to reduce the number of neighbors per vertex.

In [ ]:
A_neigh = Neighboors(A,5)
np.fill_diagonal(A_neigh, 0)

In the following graph the final network is presented taking into account the smoothing of the network.

In [ ]:
G = create_graph(A_neigh)
plot_graph(G)

In [ ]:
# Theta, k = 4, 6 #Choosen values of k and theta
# W_normal = Norm_W(W.copy(),Theta,k) 
A=A_neigh

In [ ]:
import networkx as nx
dt = [('len', float)]
A = A.view(dt)

G = nx.from_numpy_matrix(A)
pos = nx.spring_layout(G, k=0.5*1/np.sqrt(len(G.nodes())), iterations=50)
figure = pyplot.figure(figsize=(20, 10))
nx.draw(G,pos=nx.spring_layout(G,pos=pos),with_labels = True)
xyz = np.array([pos[v] for v in sorted(G)])

### Graph organization

In this section two methods are presented to reorganize the graph from the information given in the weight matrix

#### Spectral clusterin
This method makes use of the unsupervised spectral clustering algorithm in order to identify nodes that have close distances, and group them in the same set. This in order to obtain a reorganization of the network.
[Source](https://towardsdatascience.com/unsupervised-machine-learning-spectral-clustering-algorithm-implemented-from-scratch-in-python-205c87271045)

Initially Adjacency matrix W, Degree matrix D and the Laplacian matrix to obtain the eigenvalues and eigenvectors of the L matrix.

In [ ]:
import matplotlib
from pygsp import graphs

A_s=A_neigh
graph = graphs.Graph(A_s)
graph.set_coordinates(xyz)
graph.compute_fourier_basis()
e = graph.e
v = graph.U

In [ ]:
fig = plt.figure(figsize=[18, 6])
ax1 = plt.subplot(221)
plt.plot(e)
ax1.title.set_text('eigenvalues')

i = np.where(e < 20)[0]
ax2 = plt.subplot(222)
plt.plot(v[:, i[0]])
#ax2.title.set_text('first eigenvector with eigenvalue')
ax3 = plt.subplot(223)
plt.plot(v[:, i[1]])
ax3.title.set_text('second eigenvector with eigenvalue close to 0')
ax4 = plt.subplot(224)
plt.plot(v[:, i[2]])
ax4.title.set_text('third eigenvector with eigenvalue close to 0')
fig.tight_layout()

In [ ]:
def project_and_transpose(eigenvals, eigenvcts, num_ev):
    """Select the eigenvectors corresponding to the first 
    (sorted) num_ev eigenvalues as columns in a data frame.
    """
    eigenvals_sorted_indices = np.argsort(eigenvals)
    indices = eigenvals_sorted_indices[: num_ev]

    proj_df = pd.DataFrame(eigenvcts[:, indices.squeeze()])
    proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
    return proj_df

proj_df= project_and_transpose(e,v,10) #Taking the first 10 eigenvalues to make the analysis

The K-Means algorithm is applied and the inertia is calculated to choose the best possible number of clusters

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

inertias = []

k_candidates = range(1, 20)

for k in k_candidates:
    k_means = KMeans(random_state=42, n_clusters=k)
    k_means.fit(proj_df)
    inertias.append(k_means.inertia_)

From the following graph K=10 is chosen because it has an inertia very close to 0

In [ ]:
import seaborn as sns
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')

fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(x=k_candidates, y = inertias, s=80, ax=ax)
sns.lineplot(x=k_candidates, y = inertias, alpha=0.5, ax=ax)
ax.set(title='Inertia K-Means', ylabel='inertia', xlabel='k');

In [ ]:
def run_k_means(df, n_clusters):
    """K-means clustering."""
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster

def spectral_clustering(eigenvals, eigenvcts, n_clusters):
    """Spectral Clustering Algorithm."""
    proj_df = project_and_transpose(eigenvals, eigenvcts, n_clusters)
    cluster = run_k_means(proj_df, proj_df.columns.size)
    return cluster

The algorithm is executed for a number of clusters equal to 10

In [ ]:
Index_cluster= spectral_clustering(e,v,8)
index = np.argsort(Index_cluster)

In the following graph you can see the result of the clustering, where each color represents a different group. It is observed that most of the nodes that are physically close are grouped in the same set. This information is used to reorganize the graph.

In [ ]:
dt = [('len', float)]
A_s = A_s.view(dt)
G = nx.from_numpy_matrix(A_s)
pos = nx.spring_layout(G, k=0.5*1/np.sqrt(len(G.nodes())), iterations=50)
figure = pyplot.figure(figsize=(20, 10))
nx.draw(G,pos=nx.spring_layout(G,pos=pos),with_labels = True,node_color = Index_cluster, cmap = 'hsv')

In [ ]:
SortW = Sort=pd.DataFrame(data = A_neigh).iloc[index,index]
A_s=SortW.values
Grap_s=A_s

In [ ]:
fig = px.imshow(Grap_s)
fig.update_layout(
    title='Adjacent matrix')
fig.show()

In [ ]:
dt = [('len', float)]
A_s = A_s.view(dt)

G = nx.from_numpy_matrix(A_s)
pos = nx.spring_layout(G, k=0.5*1/np.sqrt(len(G.nodes())), iterations=50)
figure = pyplot.figure(figsize=(20, 10))
nx.draw(G,pos=nx.spring_layout(G,pos=pos),with_labels = True)
xyz = np.array([pos[v] for v in sorted(G)])

#### Bandwidth Reduction

The reverse Cuthill–McKee algorithm is an algorithm to permute a sparse matrix that has a symmetric sparsity pattern into a band matrix form with a small bandwidth.

In [ ]:
Theta, k = 4, 6
W_normal = Norm_W(W.copy(),Theta,k) 
A=W_normal.values
A_neigh = Neighboors(A,5)
np.fill_diagonal(A_neigh, 0)

In [ ]:
#Function
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import reverse_cuthill_mckee


def ReOrderGraph(A):
    csrMatrix = csr_matrix(A)
    perm = reverse_cuthill_mckee(csrMatrix)
    A_reorder = csrMatrix[perm, :][:, perm].toarray()
    return A_reorder,perm

def Graph_Nodes(A):
    dt = [('len', float)]
    A = A.view(dt)
    G = nx.from_numpy_matrix(A)
    pos = nx.spring_layout(G, k=0.5*1/np.sqrt(len(G.nodes())), iterations=50)
    figure = pyplot.figure(figsize=(20, 10))
    nx.draw(G,pos=nx.spring_layout(G,pos=pos),with_labels = True)
    xyz = np.array([pos[v] for v in sorted(G)])
    return xyz

In order to apply the algorithm, the "Scipy" library is used, which returns the permutation array that orders a sparse CSR or CSC matrix in Reverse-Cuthill McKee ordering. [source 2](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.reverse_cuthill_mckee.html)

In [ ]:
A_reorder,perm = ReOrderGraph(A_neigh)
xyz = Graph_Nodes(A_reorder)

In [ ]:
fig = px.imshow(A_reorder)
fig.update_layout(
    title='Adjacent matrix')
fig.show()

In [ ]:
graph = graphs.Graph(A_reorder)
graph.set_coordinates(xyz)
graph.compute_fourier_basis()
plot_graph(graph)

It is observed that with both methods a similar result is obtained, however due to simplicity the Cuthill-McKee band reduction method is chosen

In [ ]:
graph = graphs.Graph(A_reorder)
graph.set_coordinates(xyz)
graph.compute_fourier_basis()

## **Simulated Drift Implementation**


In this section We will define sinusoidal signals for all the nodes of the network to be analyzed, presenting in the vertex 0 a drift behavior from a certain instant of time. We will then analyze different characteristics of the network using the Graph Fourier Transform (GFT).

In [ ]:
matplotlib.rcParams["image.cmap"] = 'viridis'

# Constants
SPACE_GRAPH_ORDER, SPACE_KERNEL_SCALE = graph.N, 1 
TIME_GRAPH_ORDER, TIME_KERNEL_SCALE = 50, 10
dT = 10.0/TIME_GRAPH_ORDER # Time sample
LEVEL_NOISE, BREAK_TIME, STD = 0.3, 20, 3 # LN, brownian Motion (BM) start-point and SD in BM
VERTEX_D = 4 # Drift vertex

### Network in time and space

In the following graph, the two graphs to be analyzed are presented. On the one hand, we have the network of sensors obtained in the previous session, and on the other, we have a path graph which represents the time, in this case 50 different moments. Small groups of nodes are made, in order to simulate similar segmented behavior in the network


In [ ]:
# Space-time graph definition
groups = np.array([0]*21 + [1]*22)
space_time_graph = [graph, create_path_graph(TIME_GRAPH_ORDER)]

plot_graph(space_time_graph[0],groups)
plot_graph(space_time_graph[1])

In [ ]:
kernel_anima(space_time_graph, (SPACE_GRAPH_ORDER//2,TIME_GRAPH_ORDER//2), 
             windows_kernels = [SPACE_KERNEL_SCALE, TIME_KERNEL_SCALE])

### Signal definition
Now we create a function that defines the signal to be implemented on the space network, based on different space/time eigenvalues.

Let's see how each signal looks in all the vertex.

In [ ]:
sgroups = np.array([5]*21 + [34]*22)
tgroups = [6]*(TIME_GRAPH_ORDER//3) + [20]*(TIME_GRAPH_ORDER//3) + [38]*(TIME_GRAPH_ORDER - 2*(TIME_GRAPH_ORDER//3))

space_time_graph = [graph, create_path_graph(TIME_GRAPH_ORDER)]
time_series = time_space_signal_gen(space_time_graph, sgroups, tgroups, ln = LEVEL_NOISE, normalize = True)
brownian(time_series[VERTEX_D,BREAK_TIME - 1], TIME_GRAPH_ORDER - BREAK_TIME, dT, STD, out = time_series[VERTEX_D,BREAK_TIME:])

In [ ]:
import plotly.graph_objects as go
# Create traces
fig = go.Figure()
for i in range(len(time_series)):
    fig.add_trace(go.Scatter(x=list(range(len(time_series[i]))), 
                             y=time_series[i], mode='lines', name='Node'+str(i)))

fig.update_layout(
    title="Experimental signals",
    xaxis_title="Time",
    yaxis_title="Signal value")

fig.show()

The next graph shows the signal inside the space-time graph in a given time. For example, at time instant 3, it is observed that the network is divided into two sections because the signal strength is inverse for the groups created, that is, one group of nodes has a high value while the other has a low.

In [ ]:
signal_graph_anima(space_time_graph, time_series)

### GFT in time and space

In this section we present functions to calculate the graph fourier transform in time and space

To check that the desired signal was processed correctly, the signal on a particular node and its Graph Fourier Transform (GFT) can be displayed with the following code section. 

In [ ]:
gft_signal_anima(space_time_graph[1], time_series, is_graph_space = False)

From the graph above it can be seen that the signal at node 0 (where the drift occurs) most of its energy is concentrated in the low frequencies, in other words in the eigenvalues close to 0. For the other nodes, there are higher frequencies, being these on the seventh eigenvalue.

This behavior is explained having in mind that after the drift, the signal in the 0 vertex presents a constant decrease, while in the rest of sensors the sinusoidal signal continues its periosity. 

In [ ]:
gft_signal_anima(space_time_graph[0], time_series, is_graph_space = True)

From the graph above, it is observed that in general, the frequency components in time are found in greater magnitude at low levels, however they present in higher frequencies, because the signal in a moment of time varies in the network in general. After the drift in certain instants, it is observed greater power in high frequencies / eigenvalues

### Spectrogram for disjoint GFT

To have a more detailed understanding, we will first analyze the graphs separately. The following function allows to display the spectrogram of each network by selecting a specific time (for the spatial graph) or a specific vertex (for the temporal graph), depending on the case.

In [ ]:
spectogram_anima(space_time_graph[0], time_series, SKS = SPACE_KERNEL_SCALE, 
                 is_graph_space = True, limits = [0.0, 0.5], lamdba_lim = None)

In [ ]:
spectogram_anima(space_time_graph[1], time_series, SKS = TIME_KERNEL_SCALE, 
                 is_graph_space = False, limits = None, lamdba_lim = None)

* **Spatial-graph:** It is important to consider that the same signal is passed through a certain group of vertices $s(v,t_{i})$. It is observed that before the drift there is high power at low frequency in these vertices, after the drift higher frequency components are observed with high power values, corresponding in turn to the "group".

* **Time-graph:** Focusing on the signal from only one particular node $s(v_{i},t)$, it is easy to determine which vertex is faulty, since the respective spectrogram differs greatly from the other vertices, even though the signal of all should be the same. For the vertex 0, it is observed that before the drift (instant 20) the frequency component is high and after it is reduced to eigenvalues close to zero, behavior that is explained because after the drift the signal decreases progressively 

### Spectrogram for joint GFT (JFT)
Now we will enter to observe the properties of the space-time joint spectrogram. This is possible thanks to the following function. Since the joint spectrogram is in principle a two-dimensional array for each possible combination in space and time, we have as a result an array in $\mathbb{R}^4$, being useful to indicate a specific vertex and instant, looking for interpretability.

In [ ]:
kernels, joint_spectogram = JFT_anima(space_time_graph, time_series, [SPACE_KERNEL_SCALE, TIME_KERNEL_SCALE],
                                     lspace_lim = None, ltime_lim = None)

Taking into account that the joint spectrogram estimates different spectrograms, each of them centered on the combination ($v_i$, $t_j$), the analysis of the results should be done for each possible combination, arriving at the following statements:

1.  As explained above, the process occurs at "low frequencies" for any possible event, in time, which directly affects the joint spectrogram, being only possible to see non-zero magnitudes in the upper left region (low orders of eigenvalues in time).
2.  The signal $s(v_i,t)$ has more frequency components compared to the signal $s(v,t_i)$, being possible to observe in the spectrogram for any combination. 


### Drift detection

In order to detect the drift concept, we will use one technique to detect outliers. For this, we will perform the subtraction of sub-spectrograms centered on the pair $(v_i, t_j)$ and plot the result, trying to find the most changing sub-spectrogram.

In [ ]:
SPACE_WINDOW, TIME_WINDOW = 0, 2
filename = './data/drift_joint_sim.csv'

In [ ]:
dist_matrix = plot_dist_matrix(joint_spectogram.copy(), SPACE_WINDOW, TIME_WINDOW, filename = filename, 
                               norm_each_sg = False, overwrite = False,)

In [ ]:
dist_matrix = plot_dist_matrix(joint_spectogram.copy(), SPACE_WINDOW, TIME_WINDOW, filename = filename, 
                               norm_each_sg = False, overwrite = False,
                               vlist = range(VERTEX_D-1,VERTEX_D+2),)

## **Real signal analysis**

By observing a first view of the behavior of a signal that presents drift in one of its nodes, in order to understand its spectral representation, we can begin to analyze the signals captured by the sensor network. To do this, we must take a reading of the gas to be analyzed: PM, which is a measure of how polluted the air is at a location. We will then build a matrix containing the signal from each sensor per hour, from August 28 to October 28, 2019.

### Data read
Let's start with the reading of the signals, contained in the directory `air-polution-sensor/data/sensorv2/`.

In [ ]:
def readFileList(file_directory, ext = 'csv'): # Read a valid json-files inside folder / independenly file
    files_list = []
    if os.path.isdir(file_directory): # Return files with 'json' extension
        for root_path, _, files_name in os.walk(file_directory):
            files_list += [os.path.join(root_path, x) for x in files_name if x.split(".")[-1].lower() == ext.lower()]
    elif file_directory.split('.')[-1] == ext: files_list = [file_directory] # Return file inside of list
    return files_list

def sensor_read_signal(signal_name = "PM2_AVG"): # Read all signals and join in one matrix
    signals_list = readFileList("./data/data_filled")
    signals = None
    for signal_file in signals_list:
        new_col_name = os.path.split(signal_file)[-1].replace("v2.csv","").replace("sensor","")
        new_col_name = int(new_col_name) # Only keep sensor id
        sig = pd.read_csv(signal_file).rename(columns = {signal_name: new_col_name}) # Read signal
        sig["Date"] = pd.to_datetime(sig["Date"] + " " + sig["Hour"].astype(str) + ":00:00")
        sig.set_index("Date", inplace = True)
        sig=sig.loc['2019-10-01':'2019-10-03'] #For drift
        #sig=sig.loc['2019-10-01':'2019-12-30']
        # Merge by date
        if signals is None: signals = sig[[new_col_name]].copy()
        else: signals = signals.merge(sig[[new_col_name]], copy = False, left_index = True, right_index = True)
    signals = signals.sort_index().T
    
    # Fill empty sensors with NaN values
    #start, end = signals.index.min(), signals.index.max() # Find empty frames and fill in
    # start, end = 1,50
    # empty_sensors = sorted(set(range(start, end + 1)).difference(signals.index.values))
    # if len(empty_sensors) > 0: 
    #     signals = signals.reindex(signals.index.to_list() + empty_sensors).fillna(0.0) # Comment fillna(0.0) to return NaN values
    print(signals.index.sort_values())
    return signals.sort_index().values # Comment .values to return dataframe

time_sensors_series = sensor_read_signal()
print(time_sensors_series.shape)
#time_sensors_series#.head(30)

In [ ]:
sig = pd.read_csv("./data/data_filled/sensor13v2.csv").rename(columns = {"PM2_AVG": 1}) # Read signal
sig["Date"] = pd.to_datetime(sig["Date"] + " " + sig["Hour"].astype(str) + ":00:00")
sig["Day"] = sig["Date"].dt.date
sig["date_2"] = sig["Date"]
sig.set_index("Date", inplace = True)

In [ ]:
sig=sig.loc['2019-10-01':'2019-10-03']
fig = px.line(sig, x="date_2", y=1)
fig.show()

In [ ]:
matplotlib.rcParams["image.cmap"] = 'viridis'

# Constants
SPACE_GRAPH_ORDER, SPACE_KERNEL_SCALE = graph.N, 3 #40, 3
TIME_GRAPH_ORDER, TIME_KERNEL_SCALE = time_sensors_series.shape[1], 3

In [ ]:
space_time_graph = [graph, create_path_graph(TIME_GRAPH_ORDER)]

In [ ]:
perm

In [ ]:
time_sensors_series=time_sensors_series[perm]

In [ ]:
t = np.linspace(0, TIME_GRAPH_ORDER*dT, TIME_GRAPH_ORDER)
t = np.ones((SPACE_GRAPH_ORDER,1))*t[None]
# plot_curves(t, time_sensors_series, xlabel = "time", ylabel = "Vertex-magnitud")

In [ ]:
import plotly.graph_objects as go
# Create traces
fig = go.Figure()
for i in range(len(time_sensors_series)):
  fig.add_trace(go.Scatter(x=t[i], y=time_sensors_series[i],
                    mode='lines',
                    name='Node'+str(i)))

fig.update_layout(
    title="Study window",
    xaxis_title="Time instant",
    yaxis_title="PM 2"

)

fig.show()

In [ ]:
space_time_graph[1].N

In [ ]:
signal_graph_anima(space_time_graph, time_sensors_series)

In [ ]:
gft_signal_anima(space_time_graph[1], time_sensors_series, is_graph_space = True, x_lim = [0,50])

In [ ]:
gft_signal_anima(space_time_graph[0], time_sensors_series, is_graph_space = False)

### DFT analysis

#### Spatial-graph spectogram

In [ ]:
spectogram_anima(space_time_graph[0], time_sensors_series, SKS = SPACE_KERNEL_SCALE, is_graph_space = True, 
                 limits = [0.0, 3000])

#### Time-graph spectrogram

In [ ]:
spectogram_anima(space_time_graph[1], time_sensors_series, SKS = TIME_KERNEL_SCALE, is_graph_space = False, 
                 limits = None, lamdba_lim = [0,500])

In [ ]:
kernels, joint_spectogram = JFT_anima(space_time_graph, time_sensors_series, [SPACE_KERNEL_SCALE, TIME_KERNEL_SCALE],
                                     lspace_lim = None, ltime_lim = None)

### Drift detection

In [ ]:
SPACE_WINDOW, TIME_WINDOW = 1, 10
filename = './data/drift_joint_real.csv'

In [ ]:
plot_norm_matrix(joint_spectogram, SPACE_WINDOW, TIME_WINDOW)

In [ ]:
# dist_matrix = plot_dist_matrix(joint_spectogram.copy(), SPACE_WINDOW, TIME_WINDOW, filename = filename, overwrite = False,
#                                xlim = [170,270], ylim = [170,270],) #image_save = './dist_real.png',)
# dist_matrix.columns[170:270]
# dist_matrix.columns[170:270]

## ***Anexos: GitHub-Colab connection***
Here, some commands to upload/save the github respository

In [ ]:
''' Function definitions'''
# Git pull
def git_pull(repo_pwd, show_current_branch = False, make_commit = False): # Only for colab space work
    global user_git, email_git
    import sys
    IN_COLAB = 'google.colab' in sys.modules
    if IN_COLAB:
        from google.colab import drive
        drive.mount('/content/gdrive')

        %cd "$repo_pwd"
        # !git config --list
        if show_current_branch: 
            !git branch 
        if make_commit:
            if "user_git" not in globals(): user_git = input("User github?: ")
            if "email_git" not in globals(): email_git = input("Email github?: ") 
            !git config --global user.email $email_git
            !git config --global user.name $user_git
            !git commit -am "Updating in colab"
        !git pull
        !git status
    else:
        print("[INFO] You are not in collaboration, nothing has been done.")

# Git push
def git_push(repo_pwd): # Only for colab space work
    global user_git, email_git
    import sys
    IN_COLAB = 'google.colab' in sys.modules
    if IN_COLAB:
        from google.colab import drive
        import getpass
        drive.mount('/content/gdrive')

        %cd "$repo_pwd"
        if "user_git" not in globals(): user_git = input("User github?: ")
        if "email_git" not in globals(): email_git = input("Email github?: ")

        # Password login
        try: 
            pwd_git = getpass.getpass(prompt='{} github password: '.format(user_git)) 
        except Exception as error: 
            print('ERROR', error) 

        # Upload from every where
        origin_git = !git config --get remote.origin.url
        origin_git = origin_git[0].replace("https://","https://{}:{}@".format(user_git,pwd_git))

        !git config --global user.email "$email_git"
        !git config --global user.name "$user_git"
        !git status

        x = " "
        while x.lower() != "y" and x.lower() != "n": x = input("Continue?...[y/n]: ")

        if x.lower() == "y":
            com_message = input("Enter the commit message: ")
            !git add .
            !git commit -am "$com_message"
            !git push "$origin"
            !git status
    else:
        print("[INFO] You are not in collaboration, nothing has been done.")

In order to execute the functions, please unlock the respective function

In [ ]:
# git_pull(repo_pwd, show_current_branch = False, make_commit = True)
# git_push(repo_pwd)